In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mchammer import MonteCarlo
from mchammer.ensembles import GrandCanonicalEnsemble
from mchammer.observers import EnergyObserver, CompositionObserver
from icet import ClusterExpansion
from ase.io import write  # For writing structures to files
from ase.io import read as ase_read

# Load the cluster expansion model
ce = ClusterExpansion.load("ce_jochen")

# Define the initial structure (e.g., a Lithium-based system)
initial_structure = bulk("Li", "fcc", a=4.1, cubic=True)  # TODO: structure

# Set up observers to track energy and composition
energy_observer = EnergyObserver()
composition_observer = CompositionObserver()

# Define the lithium chemical potential (this can be tweaked for lithium content control)
lithium_chemical_potential = -1.0  # Adjust as needed to influence Li content

# Define temperature range (1000K to 300K, in steps of 50K)
temperatures = np.arange(1000, 300 - 1, -50)

# Storage for results
energy_per_temp = []
composition_per_temp = []

# Perform Monte Carlo simulation for each temperature (1 step per temperature)
for temp in temperatures:
    # Set up the grand-canonical ensemble (for tweaking Li content via chemical potential)
    ensemble = GrandCanonicalEnsemble(
        cluster_expansion=ce,
        initial_structure=initial_structure,
        temperature=temp,
        chemical_potentials={"Li": lithium_chemical_potential}  # Control Li content
    )

    # Initialize the Monte Carlo simulation
    mc_simulation = MonteCarlo(
        ensemble=ensemble,
        observers=[energy_observer, composition_observer],  # Track energy and composition
    )

    # Run just one Monte Carlo step at this temperature
    mc_simulation.run(steps=1)

    # Store the final energy and composition for the current temperature
    final_energy = energy_observer.energies[-1]
    final_composition = composition_observer.compositions[-1]["Li"]  # Get Li content

    energy_per_temp.append(final_energy)
    composition_per_temp.append(final_composition)

    # Save the current structure to a file
    # We'll name the files based on the temperature (e.g., "structure_1000K.xyz")
    filename = f"structure_{int(temp)}K.xyz"
    write(filename, ensemble.structure)  # Save the structure in XYZ format

    # Reset observers for the next temperature run (optional, since 1 step per temp)
    energy_observer.reset()
    composition_observer.reset()

# Plot results: Energy and Lithium composition vs. Temperature

# Plot energy vs. temperature
plt.figure()
plt.plot(temperatures, energy_per_temp, label="Energy")
plt.xlabel("Temperature (K)")
plt.ylabel("Energy (eV)")
plt.title("Energy vs Temperature")
plt.grid(True)
plt.show()

# Plot composition (Li content) vs. temperature
plt.figure()
plt.plot(temperatures, composition_per_temp, label="Lithium Content", color='orange')
plt.xlabel("Temperature (K)")
plt.ylabel("Lithium Content (Fraction)")
plt.title("Lithium Content vs Temperature")
plt.grid(True)
plt.show()

# Optionally, print the final lithium content at 300K
print(f"Final lithium content at 300K: {composition_per_temp[-1]}")
